# Neptune + PyTorch Lightning

## Introduction

This guide will show you how to:

* Create a `NeptuneLogger()`,
* Log training metrics to Neptune using `NeptuneLogger()`.

## Before you start

This notebook example lets you try out Neptune as an anonymous user, with zero setup.

* If you are running the notebook on your local machine, you need to have [Python](https://www.python.org/downloads/) and [pip](https://pypi.org/project/pip/) installed.
* If you want to see the example recorded to your own workspace instead:
    * Create a Neptune account → [Take me to registration](https://neptune.ai/register)
    * Create a Neptune project that you will use for tracking metadata → [ Tell me more about projects](https://docs.neptune.ai/administration/projects)

## Install Neptune and dependencies

In [ ]:
! pip install --quiet neptune-client >= 0.11.0 pytorch-lightning >= 1.5.0 scikit-learn scikit-plot == 0.3.7 matplotlib == 3.4.2

## Import libraries

In [ ]:
import os

import numpy as np
import torch
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.loggers import NeptuneLogger
from sklearn.metrics import accuracy_score
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST

## Define hyper-parameters

In [ ]:
params = {
    "batch_size": 32,
    "lr": 0.007,
    "max_epochs": 15,
}

## Define LightningModule and DataLoader

In [ ]:
# (neptune) define LightningModule with logging (self.log)
class MNISTModel(LightningModule):
    def __init__(self):
        super().__init__()
        self.l1 = torch.nn.Linear(28 * 28, 10)

    def forward(self, x):
        return torch.relu(self.l1(x.view(x.size(0), -1)))

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log("metrics/batch/loss", loss, prog_bar=False)

        y_true = y.cpu().detach().numpy()
        y_pred = y_hat.argmax(axis=1).cpu().detach().numpy()
        acc = accuracy_score(y_true, y_pred)
        self.log("metrics/batch/acc", acc)

        return {"loss": loss, "y_true": y_true, "y_pred": y_pred}

    def training_epoch_end(self, outputs):
        loss = np.array([])
        y_true = np.array([])
        y_pred = np.array([])
        for results_dict in outputs:
            loss = np.append(loss, results_dict["loss"])
            y_true = np.append(y_true, results_dict["y_true"])
            y_pred = np.append(y_pred, results_dict["y_pred"])
        acc = accuracy_score(y_true, y_pred)
        self.log("metrics/epoch/loss", loss.mean())
        self.log("metrics/epoch/acc", acc)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=params["lr"])


# init model
mnist_model = MNISTModel()

# init DataLoader from MNIST dataset
train_ds = MNIST(os.getcwd(), train=True, download=True, transform=transforms.ToTensor())
train_loader = DataLoader(train_ds, batch_size=params["batch_size"])

## Create NeptuneLogger

In [ ]:
neptune_logger = NeptuneLogger(
    api_key="ANONYMOUS",
    project="common/pytorch-lightning-integration",
    tags=["simple", "showcase"],
    log_model_checkpoints=False,
)

## Initialize a trainer and pass neptune_logger

In [ ]:
# (neptune) initialize a trainer and pass neptune_logger
trainer = Trainer(
    logger=neptune_logger,
    max_epochs=params["max_epochs"],
)

## Log hyper-parameters to the run

In [ ]:
neptune_logger.log_hyperparams(params=params)

## Train the model log metadata to the Neptune run

In [ ]:
trainer.fit(mnist_model, train_loader)

## Stop logging

<font color=red>**Warning:**</font><br>
Once you are done logging, you should stop the run using the `stop()` method, which is a method of the run object. The run object itself is part of the `neptune_logger`.
This is needed only while logging from a notebook environment. While logging through a script, Neptune automatically stops tracking once the script has completed execution.

In [ ]:
neptune_logger.experiment.stop()

## Analyze logged metadata in the Neptune UI

Go to the run link and explore metadata (metrics, hparams, model checkpoint) that were logged to the run in Neptune.

Link should look like this: https://app.neptune.ai/o/common/org/pytorch-lightning-integration/e/PTL-18/dashboard/simple-6ff16e4c-c529-4c63-b437-dfb883131793